In [1]:
import pandas as pd
import numpy as np
import math
import time
import datetime
import statsmodels.api as sm

from NSCOracle import *
from NSCDataProcess import * 
from clickhouse_driver import Client


import warnings
warnings.filterwarnings("ignore")




/home/developer/anaconda3/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from Jwfunctions import *

In [5]:
def get_weights(m,T,tn):
    w = m**(1/T)
    """生成weight序列"""
    w_list = np.array([w**(tn-1-i) for i in range(tn)])
    return w_list

In [7]:
dt = '20220328'
end_date = dt

In [19]:
dt_list =  get_Njyr(dt,252)

In [11]:
sql_ = """

with tmp_innercode as (
SELECT innercode,secucode 
FROM secumain
WHERE SECUMARKET IN (83,90)
AND secucategory = 1 
AND LISTEDSTATE IN (1)
AND secucode IN ('000001','002001','003000','300001','301000','600000','601001','605001','688001')
group by innercode ,secucode 
ORDER BY secucode )
"""
df_innercode = get_data_from_oracle(qt_cursor,sql_)

df_innercode['innercode'] = df_innercode['innercode'].astype(str)

In [89]:
def GetHq(qt_cursor,innercode_dict ,start_date,end_date):
    sql_ = """
    select to_char(TradingDay,'yyyymmdd') as tradingday,
        innercode,
        changepct,
        turnovervalue,
        turnoverrate,
        BetaHS300Index as beta_1y,-- Beta值(相对沪深300,一年)
        ChangePCT/100  as daily_return,-- 涨跌幅(%)//个股每日收益率
        TurnoverRateR6M  as decl_dr_wgt_mean_6_month, -- 六个月换手率(%)
        AlphaHS300Index as historical_alpha  
    from QT_Performance
    WHERE  innercode IN ('{}') 
    and to_char(tradingday,'yyyymmdd') between '{}' and '{}'
    """.format("','".join(list(innercode_dict.keys())),start_date,end_date)
    #"','".join(list(df_innercode['innercode'].unique())),"','".join(dt_list)
    df_hq =  get_data_from_oracle(qt_cursor,sql_)
    df_hq['innercode'] = df_hq['innercode'].astype(str)
    innercode_dict = df_innercode.groupby(['innercode'])['secucode'].apply(list)
    innercode_dict = {k:v[0] for k,v in innercode_dict.items()}
    df_hq['secucode'] = df_hq['innercode'].map(innercode_dict)
    return df_hq

In [24]:
sql_ = """
select to_char(TradingDay,'yyyymmdd') as tradingday,
    innercode,
    changepct,
    turnovervalue,
    turnoverrate,
    BetaHS300Index as beta_1y,-- Beta值(相对沪深300,一年)
    ChangePCT/100  as daily_return,-- 涨跌幅(%)//个股每日收益率
    TurnoverRateR6M  as decl_dr_wgt_mean_6_month, -- 六个月换手率(%)
    AlphaHS300Index as historical_alpha  
from QT_Performance
WHERE  innercode IN ('{}') 
and to_char(tradingday,'yyyymmdd') in ('{}')
""".format("','".join(list(df_innercode['innercode'].unique())),"','".join(dt_list))
df_hq =  get_data_from_oracle(qt_cursor,sql_)

df_hq['innercode'] = df_hq['innercode'].astype(str)
innercode_dict = df_innercode.groupby(['innercode'])['secucode'].apply(list)
innercode_dict = {k:v[0] for k,v in innercode_dict.items()}
df_hq['secucode'] = df_hq['innercode'].map(innercode_dict)

In [39]:
df_hq.groupby(['secucode'])['changepct'].count()

secucode
000001    252
002001    252
003000    249
300001    252
301000    200
600000    252
601001    252
605001    249
Name: changepct, dtype: int64

In [33]:
df_hq.head()
df_hq.isnull().sum()

tradingday                    0
innercode                     0
changepct                     0
turnovervalue                 0
turnoverrate                  0
beta_1y                     277
daily_return                  0
decl_dr_wgt_mean_6_month      0
historical_alpha            277
secucode                      0
dtype: int64

In [34]:
"""
annualized_traded_value_ratio: 日交易份额比率（换手率）进行加权求和，时间窗口252交易日，半衰期63交易日,
"""
col = 'annualized_traded_value_ratio'
N = 252
turnoverrate = stack2dataframe(df_hq,'tradingday','secucode','turnoverrate')
w_list = get_weights(0.5,63,252)
weights = turnoverrate.copy()
weights.loc[:,:] = np.repeat(
    w_list.reshape(252,1),len(turnoverrate.columns),axis = 1)
annualized_traded_value_ratio = (weights * turnoverrate ).rolling(252,min_periods = 1).sum()/np.sum(weights)
annualized_traded_value_ratio = dict(annualized_traded_value_ratio.loc[dt,:])
annualized_traded_value_ratio

In [51]:
"""
average_turnoverrate_1m: 计算过去21个交易日的换手率之和,
average_turnoverrate_1q: 计算过去63个交易日的换手率之和,
average_turnoverrate_1yr: 计算过去252个交易日的换手率之和,
"""
N = 21
dt_list_ = get_Njyr(end_date,N)

turnoverrate = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)],'tradingday','secucode','turnoverrate')
average_turnoverrate_1m = np.log(np.nansum(turnoverrate/100,axis = 0))
average_turnoverrate_1m = {k:v for k,v in zip(turnoverrate.columns,average_turnoverrate_1m)}
average_turnoverrate_1m

{'000001': -2.021693523912671,
 '002001': -1.3760747595926315,
 '003000': -2.0074037107907134,
 '300001': -1.3587803665198503,
 '301000': -0.9783629634722172,
 '600000': -3.7936179410495248,
 '601001': -0.3706782484332818,
 '605001': -1.005143803743156}

In [53]:
N = 63
dt_list_ = get_Njyr(end_date,N)
turnoverrate = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','turnoverrate')
stom_list = []
for i in range(0,N,21):
    stom = np.log(np.nansum(turnoverrate[turnoverrate.index[i]:turnoverrate.index[i+20]]/100,axis = 0))
    stom_list.append(stom)
average_turnoverrate_1q = np.log(np.nanmean(np.exp(np.stack(stom_list,axis = 0)),axis = 0))
average_turnoverrate_1q = {k:v for k,v in zip(turnoverrate.columns,average_turnoverrate_1q)}
average_turnoverrate_1q


{'000001': -2.042939906486159,
 '002001': -1.4791423745451033,
 '003000': -1.5272129721784309,
 '300001': -1.3149788557032958,
 '301000': -0.9111489733994419,
 '600000': -3.6990510094354327,
 '601001': -0.34561301991311777,
 '605001': -1.543971696738222}

In [54]:
N = 252
dt_list_ = get_Njyr(end_date,N)
turnoverrate = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','turnoverrate')
stom_list = []
for i in range(0,N,21):
    stom = np.log(np.nansum(turnoverrate[turnoverrate.index[i]:turnoverrate.index[i+20]]/100,axis = 0))
    stom_list.append(stom)
average_turnoverrate_1yr = np.log(np.nanmean(np.exp(np.stack(stom_list,axis = 0)),axis = 0))
average_turnoverrate_1yr = {k:v for k,v in zip(turnoverrate.columns,average_turnoverrate_1yr)}
average_turnoverrate_1yr

{'000001': -2.2571210892889866,
 '002001': -1.6959653156488423,
 '003000': 0.022440647925981324,
 '300001': -0.813441993088371,
 '301000': -0.20575386140617163,
 '600000': -3.568294421793349,
 '601001': -0.4720419719220559,
 '605001': -1.8233817239240635}

In [55]:
"""
daily_std_dec: 日收益率在过去252个交易日的波动率，半衰期42个交易日,
"""
N = 252
dt_list_ = get_Njyr(end_date,N)
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')
changepct = changepct.fillna({k:v for k,v in zip(changepct.columns,np.nanmean(changepct,axis = 0))})


"""
dastd = sqrt(sum(wi*(ri-avg(ri))^2))
"""
w = (1/2)**(1/42)
tn = 252
"""生成weight序列"""
w_list = np.array([w**(tn-1-i) for i in range(tn)])
weights = np.repeat(w_list.reshape(N,1),len(changepct.columns),axis = 1)
daily_std_dec = (np.nansum((changepct.values - np.nanmean(changepct,axis = 0))**2  * weights,axis = 0)/\
np.nansum(weights,axis = 0))**0.5

daily_std_dec = {k:v for k,v in zip(changepct.columns,daily_std_dec)}
daily_std_dec

In [58]:
"""hist_sigma: 在计算BETA所进行的时间序列回归中，取回归残差收益率的波动率,"""

sql_ = """
SELECT 
       to_char(tradingday,'yyyymmdd') AS tradingday ,
       secucode , 
       prevcloseprice,closeprice,
       changepct -- (closeprice(t) /closeprice(t-1)-1)*100
FROM t_qt_indexquote
WHERE secucode = '000300' 
AND to_char(tradingday,'yyyymmdd')  >={} and to_char(tradingday,'yyyymmdd') <={}
ORDER BY tradingday  
""".format(dt_list[0],dt_list[-1])
df_300 = get_data_from_oracle(cursor,sql_) 
index_pct_list = np.array(df_300.sort_values('tradingday')['changepct'])/100
index_pct = np.repeat(index_pct_list.reshape(N,1),len(changepct.columns),axis = 1)

In [60]:
x = changepct.values
y = index_pct_list.copy()
Y = sm.add_constant(y)
model = sm.WLS(x,Y,w_list)
result = model.fit()
# alpha_ ,beta_  = result.params
resid_ = result.resid
hist_sigma = np.nanstd(resid_,axis = 0)
hist_sigma = {k:v for k,v in zip(changepct.columns,hist_sigma)}
hist_sigma


{'000001': 0.017893675314049298,
 '002001': 0.01910633931922418,
 '003000': 0.024050194917631173,
 '300001': 0.030466434581180396,
 '301000': 0.035249040600698896,
 '600000': 0.008717085903893366,
 '601001': 0.040316691781125374,
 '605001': 0.015008810537080784}

In [61]:
"""
beta_1y: 
股票收益率 对沪深300收益率，进行时间序列回归，取回归系数，回归时间窗口为252个交易日，半衰期63个交易日,
historical_alpha: 
股票收益率 对沪深300收益率 进行时间序列回归，取截距项，回归时间窗口为252个交易日，半衰期63个交易日,
"""

changepct = changepct.fillna({k:v for k,v in zip(changepct.columns,np.nanmean(changepct,axis = 0))})
w = 0.5**(1/63)
tn = len(changepct.index)
"""生成weight序列"""
w_list = np.array([w**(tn-1-i) for i in range(tn)])

x = changepct.values
y = index_pct_list.copy()
Y = sm.add_constant(y)
model = sm.WLS(x,Y,w_list)
result = model.fit()
alpha_ ,beta_  = result.params

historical_alpha = {k:v for k,v in zip(changepct.columns,alpha_)}
beta_1y = {k:v for k,v in zip(changepct.columns,beta_)}

In [62]:
beta_1y

{'000001': 1.1308287545422457,
 '002001': 0.846038453982691,
 '003000': 0.7762213322211399,
 '300001': 0.9674218446019972,
 '301000': 0.6837362095677564,
 '600000': 0.4536700367423684,
 '601001': 0.9338521063680909,
 '605001': 0.6284408884777584}

In [63]:
historical_alpha

{'000001': -6.287594354496926e-05,
 '002001': 0.0019074861342970565,
 '003000': -0.0006617185442121849,
 '300001': -0.0010772392820739478,
 '301000': -0.00080630471213623,
 '600000': -0.0005610853236739063,
 '601001': 0.00780394285998786,
 '605001': -0.0003506231341021028}

In [65]:
"""
decl_dr_wgt_mean_12_month: 最近n个月以每日换手率为权重，计算每日收益率的算术平均值，n=12，换手率随时间衰减,
decl_dr_wgt_mean_6_month: 最近n个月以每日换手率为权重，计算每日收益率的算术平均值，n=6，换手率随时间衰减,
"""

N = 6*20
dt_list_ = get_Njyr(end_date,N)

turnoverrate = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','turnoverrate')
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')
w = (1/math.e)**(1/(120*4))
tn = N

"""生成weight序列"""
w_list = np.array([w**(tn-1-i) for i in range(tn)])

weigths = turnoverrate.values * np.repeat(w_list.reshape(N,1),len(changepct.columns),axis =1)
decl_dr_wgt_mean_6_month = np.nansum(weigths * changepct.values ,axis = 0)/np.nansum(weigths,axis = 0)
decl_dr_wgt_mean_6_month = {k:v for k,v in zip(changepct.columns,decl_dr_wgt_mean_6_month)}
decl_dr_wgt_mean_6_month

{'000001': 0.0006283818845288218,
 '002001': 0.004342929097922614,
 '003000': 0.004086400450708994,
 '300001': -0.0035024608966566914,
 '301000': 0.0009508548644631654,
 '600000': -0.001385929898511407,
 '601001': 0.007452750552969538,
 '605001': 0.004757740497299545}

In [66]:
N = 12*20
dt_list_ = get_Njyr(end_date,N)

turnoverrate = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','turnoverrate')
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')

w = (1/math.e)**(1/(N*4))
tn = N
"""生成weight序列"""
w_list = np.array([w**(tn-1-i) for i in range(tn)])
weigths = turnoverrate.values * np.repeat(w_list.reshape(N,1),len(changepct.columns),axis =1)
decl_dr_wgt_mean_12_month = np.nansum(weigths * changepct.values ,axis = 0)/np.nansum(weigths,axis = 0)
decl_dr_wgt_mean_12_month = {k:v for k,v in zip(changepct.columns,decl_dr_wgt_mean_12_month)}
decl_dr_wgt_mean_12_month

{'000001': 0.0005090151018571338,
 '002001': 0.0036970640731225184,
 '003000': 0.002047738967252607,
 '300001': 0.00245675747841816,
 '301000': 0.01791843754347612,
 '600000': -0.0017724035775605006,
 '601001': 0.011055724587886952,
 '605001': 0.00072779618677351}

In [67]:
"""
dr_abs_mean_val_mean_12m: 过去12月收益率绝对值均值/成交额均值,
dr_abs_mean_val_mean_1m: 过去1月收益率绝对值均值/成交额均值,
dr_abs_mean_val_mean_3m: 过去3月收益率绝对值均值/成交额均值,
dr_abs_mean_val_mean_6m: 过去6月收益率绝对值均值/成交额均值,
"""
N = 12*20
dt_list_ = get_Njyr(end_date,N)

turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','turnovervalue')
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')


dr_abs_mean_val_mean_12m = np.nanmean(abs(changepct),axis = 0)*100000/np.nanmean(turnovervalue,axis = 0)
dr_abs_mean_val_mean_12m = {k:v for k,v in zip(changepct.columns,dr_abs_mean_val_mean_12m)}
dr_abs_mean_val_mean_12m

{'000001': 0.009125419421744397,
 '002001': 0.023108996916431406,
 '003000': 0.5278928648262851,
 '300001': 0.04069264496705553,
 '301000': 0.3941392329922336,
 '600000': 0.019384989265213336,
 '601001': 0.0584927191360096,
 '605001': 0.8122255896904516}

In [69]:
N = 1*20
dt_list_ = get_Njyr(end_date,N)

turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','turnovervalue')
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')

dr_abs_mean_val_mean_1m = np.nanmean(abs(changepct),axis = 0)*100000/np.nanmean(turnovervalue,axis = 0)
dr_abs_mean_val_mean_1m = {k:v for k,v in zip(changepct.columns,dr_abs_mean_val_mean_1m)}
dr_abs_mean_val_mean_1m

{'000001': 0.013112081147248477,
 '002001': 0.01883043741003612,
 '003000': 1.5982531238837638,
 '300001': 0.08953292473398038,
 '301000': 1.0609468737082715,
 '600000': 0.04409424348274139,
 '601001': 0.04881794456084108,
 '605001': 0.678023458523623}

In [70]:
N = 3*20
dt_list_ = get_Njyr(end_date,N)

turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','turnovervalue')
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')

dr_abs_mean_val_mean_3m = np.nanmean(abs(changepct),axis = 0)*100000/np.nanmean(turnovervalue,axis = 0)
dr_abs_mean_val_mean_3m = {k:v for k,v in zip(changepct.columns,dr_abs_mean_val_mean_3m)}
dr_abs_mean_val_mean_3m

{'000001': 0.00897096065141823,
 '002001': 0.021374791728223423,
 '003000': 0.8654677963205384,
 '300001': 0.06743182794569846,
 '301000': 0.9500568602287479,
 '600000': 0.027316898579326617,
 '601001': 0.04626549373992141,
 '605001': 0.8003022347106362}

In [71]:
N = 6*20
dt_list_ = get_Njyr(end_date,N)

turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','turnovervalue')
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')

dr_abs_mean_val_mean_6m = np.nanmean(abs(changepct),axis = 0)*100000/np.nanmean(turnovervalue,axis = 0)
dr_abs_mean_val_mean_6m = {k:v for k,v in zip(changepct.columns,dr_abs_mean_val_mean_6m)}
dr_abs_mean_val_mean_6m

{'000001': 0.008082116296062597,
 '002001': 0.0229549152783919,
 '003000': 0.6779020610097364,
 '300001': 0.04292310463105786,
 '301000': 0.4708081999593802,
 '600000': 0.024012927775587587,
 '601001': 0.049844805354915284,
 '605001': 0.8747892686418668}

In [72]:
"""
turn_val_std_mean_12m: 过去12月成交额标准差/均值,
turn_val_std_mean_1m: 过去1月成交额标准差/均值,
turn_val_std_mean_3m: 过去3月成交额标准差/均值,
turn_val_std_mean_6m: 过去6月成交额标准差/均值,
"""
N = 12*20
dt_list_ = get_Njyr(end_date,N)
turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','turnovervalue')
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')


turn_val_std_mean_12m = np.nanstd(turnovervalue,ddof = 1,axis = 0) / np.nanmean(turnovervalue,axis = 0)
turn_val_std_mean_12m = {k:v for k,v in zip(changepct.columns,turn_val_std_mean_12m)}
turn_val_std_mean_12m

{'000001': 0.4080552109562095,
 '002001': 0.5698566016092355,
 '003000': 1.0469831690549392,
 '300001': 0.6979091767445841,
 '301000': 1.4927281233098613,
 '600000': 0.4401325423432597,
 '601001': 0.5639194099780805,
 '605001': 1.2284392243168054}

In [73]:
N = 1*20
dt_list_ = get_Njyr(end_date,N)
turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','turnovervalue')
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')


turn_val_std_mean_1m = np.nanstd(turnovervalue,ddof = 1,axis = 0) / np.nanmean(turnovervalue,axis = 0)
turn_val_std_mean_1m = {k:v for k,v in zip(changepct.columns,turn_val_std_mean_1m)}
turn_val_std_mean_1m

{'000001': 0.3325229880269727,
 '002001': 0.2536468637808199,
 '003000': 0.27251531300550846,
 '300001': 0.3238209156252421,
 '301000': 0.22856559022378836,
 '600000': 0.3553489535442144,
 '601001': 0.23705582467711123,
 '605001': 1.5259907044648422}

In [74]:
N = 3*20
dt_list_ = get_Njyr(end_date,N)
turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','turnovervalue')
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')


turn_val_std_mean_3m = np.nanstd(turnovervalue,ddof = 1,axis = 0) / np.nanmean(turnovervalue,axis = 0)
turn_val_std_mean_3m = {k:v for k,v in zip(changepct.columns,turn_val_std_mean_3m)}
turn_val_std_mean_3m

{'000001': 0.3784200732436235,
 '002001': 0.464598958540541,
 '003000': 0.7586386959301195,
 '300001': 0.4244294266822615,
 '301000': 0.3558701870002229,
 '600000': 0.3720256109650311,
 '601001': 0.41053061414925723,
 '605001': 1.631447218738724}

In [75]:
N = 6*20
dt_list_ = get_Njyr(end_date,N)
turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','turnovervalue')
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')


turn_val_std_mean_6m = np.nanstd(turnovervalue,ddof = 1,axis = 0) / np.nanmean(turnovervalue,axis = 0)
turn_val_std_mean_6m = {k:v for k,v in zip(changepct.columns,turn_val_std_mean_6m)}
turn_val_std_mean_6m

{'000001': 0.38489834170671716,
 '002001': 0.5459435585049344,
 '003000': 0.7371504926650357,
 '300001': 0.711263857905947,
 '301000': 1.3458631264646046,
 '600000': 0.37992117579795837,
 '601001': 0.4571746474448867,
 '605001': 1.5937567535709851}

In [76]:

"""
std_12m: 过去12月日收益率标准差,
std_3m: 过去3月日收益率标准差,
std_6m: 过去6月日收益率标准差,
"""
N = 12*20
dt_list_ = get_Njyr(end_date,N)
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')
std_12m = np.nanstd(changepct,ddof = 1,axis = 0) 
std_12m = {k:v for k,v in zip(changepct.columns,std_12m*100)}
std_12m

{'000001': 2.229799843474294,
 '002001': 2.1434417753954236,
 '003000': 2.468507642057766,
 '300001': 3.2856543996537035,
 '301000': 4.002003907266789,
 '600000': 0.934041126605729,
 '601001': 4.101135140277298,
 '605001': 1.596392733941716}

In [80]:
N = 3*20
dt_list_ = get_Njyr(end_date,N)
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')
std_3m = np.nanstd(changepct,ddof = 1,axis = 0) 
std_3m = {k:v for k,v in zip(changepct.columns,std_3m*100)}
std_3m

{'000001': 2.2507178268025974,
 '002001': 2.632039597930367,
 '003000': 2.287668183558309,
 '300001': 2.462491649464748,
 '301000': 1.7913129595142578,
 '600000': 1.0855045815495075,
 '601001': 4.2548996559337295,
 '605001': 2.231211672062519}

In [81]:
N = 6*20
dt_list_ = get_Njyr(end_date,N)
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')
std_6m = np.nanstd(changepct,ddof = 1,axis = 0) 
std_6m = {k:v for k,v in zip(changepct.columns,std_6m*100)}
std_6m

{'000001': 2.053813066555046,
 '002001': 2.3390061484447164,
 '003000': 2.1640499357829834,
 '300001': 2.676683482447894,
 '301000': 2.2595140860305625,
 '600000': 0.917674861664191,
 '601001': 4.055888439161471,
 '605001': 1.840887245973329}

In [82]:
"""
holding_return_1m: 最近21个交易日的累计持仓收益率,
"""
N = 21
dt_list_ = get_Njyr(end_date,N)
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')
changepct_ = np.nancumprod(1+changepct,axis = 0)

holding_return_1m = np.log(changepct_)[-1,:] -1
holding_return_1m = {k:v for k,v in zip(changepct.columns,holding_return_1m)}
holding_return_1m

{'000001': -1.068318700996545,
 '002001': -1.0809159476711059,
 '003000': -1.0677250285077886,
 '300001': -1.0980293784521513,
 '301000': -1.0936059770986097,
 '600000': -1.0700083277246188,
 '601001': -0.7682516527851684,
 '605001': -1.052851683388241}

In [106]:
"""
relative_strength: 
1、计算非滞后的相对强度 对股票的对数收益率进行半衰指数加权求和，时间窗口252个交易日，半衰期126个交易日；
2、以11个交易日为时间窗口，滞后11个交易日，取非滞后相对强度的等权平均值,
"""

N = 252
dt_list = get_Njyr(end_date,N+11+11)
df_hq = GetHq(qt_cursor,innercode_dict,dt_list[0],dt_list[-1])

changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','daily_return')

changepct = np.log(1+ changepct)# 对数收益率
w = (1/2)**(1/126)
tn = N
"""生成weight序列"""
w_list = np.array([w**(N-1-i) for i in range(N)])
weights = np.repeat(w_list.reshape(N,1),len(changepct.columns),axis = 1)
rs_list = []
for i in range(11):
    rs = changepct.loc[changepct.index[-(252+11+i)]: changepct.index[-(12+i)]].values * weights
    rs_list.append(np.nansum(rs * 10000,axis = 0)/np.nansum(weights,axis = 0))

relative_strength = np.nanmean(np.stack(rs_list,axis = 0),axis = 0)
relative_strength = {k:v for k,v in zip(changepct.columns,relative_strength)}
relative_strength


{'000001': -19.682440651429005,
 '002001': 7.8685235127591815,
 '003000': -21.232669132981968,
 '300001': -21.558737256482573,
 '301000': -13.030785074333254,
 '600000': -8.906680852504243,
 '601001': 41.79101281988995,
 '605001': -12.345321743658722}

In [108]:
"""
long_term_relative_strength: 
1、计算非滞后的长期相对强度，对股票对数收益率进行加权求和，时间窗口1040个交易日，半衰期260个交易日；
2、滞后273个交易日，在11个交易日的时间窗口内取非滞后值等权平均值，最后取相反数,
"""
N = 1040
dt_list = get_Njyr(end_date,N+11+273)
df_hq = GetHq(qt_cursor,innercode_dict,dt_list[0],dt_list[-1])

changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list)].copy(),'tradingday','secucode','daily_return')
changepct = np.log(1+ changepct)# 对数收益率

w = (1/2)**(1/260)
"""生成weight序列"""
w_list = np.array([w**(N-1-i) for i in range(N)])

weights = np.repeat(w_list.reshape(N,1),len(changepct.columns),axis = 1)

rs_list = []
for i in range(11):
    rs = changepct.loc[changepct.index[-(N+273+i)]: changepct.index[-(273+1+i)]].values * weights
    rs_list.append(-np.nansum(rs * 10000,axis = 0)/np.nansum(weights,axis = 0))
long_term_relative_strength = np.nanmean(np.stack(rs_list,axis = 0),axis = 0)
long_term_relative_strength = {k:v for k,v in zip(changepct.columns,long_term_relative_strength)}
long_term_relative_strength

{'000001': -15.729328613417243,
 '002001': -15.67790447306279,
 '003000': -24.4393314044461,
 '300001': -17.852618787723312,
 '301000': 0.0,
 '600000': 0.6656531501248805,
 '601001': 0.8866362666404893,
 '605001': 5.5927484360581845}

In [109]:
"""
short_term_revarsal: 最近一个月的加权累积对数日收益率，时间窗口21个交易日，半衰期5个交易日，T={t-1，.....，t-n},
"""
N = 21
dt_list_ = get_Njyr(end_date,N)
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')

w = 0.5**(1/5)
w_list = np.array([w**(N-1-i) for i in range(N)])
weights = np.repeat(w_list.reshape(N,1),len(changepct.columns),axis = 1)
short_term_revarsal = np.nansum(changepct * weights,axis = 0)/ np.nansum(weights,axis=0) * 10000
short_term_revarsal = {k:v for k,v in zip(changepct.columns,short_term_revarsal)}
short_term_revarsal


{'000001': 1.379843115281811,
 '002001': -76.79788430796661,
 '003000': 2.962559453724516,
 '300001': -37.01731037365096,
 '301000': -11.562164383113029,
 '600000': -6.290889830223164,
 '601001': 199.53844929023813,
 '605001': -29.4084939436797}

In [110]:
"""
dr_abs_val_mean_log_20d: 过去20 日（每日涨跌幅绝对值/成交额）的均值取对数,
"""
N = 20
dt_list_ = get_Njyr(end_date,N)
changepct = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','daily_return')
turnovervalue = stack2dataframe(df_hq[df_hq['tradingday'].isin(dt_list_)].copy(),'tradingday','secucode','turnovervalue')

dr_abs_val_mean_log_20d = np.log(np.mean(abs(changepct)/turnovervalue*100000,axis = 0))
dr_abs_val_mean_log_20d = {k:v for k,v in zip(changepct.columns,dr_abs_val_mean_log_20d)}
dr_abs_val_mean_log_20d

{'000001': -4.376959965397654,
 '002001': -3.9569393547881426,
 '003000': 0.36988736193883925,
 '300001': -2.486404503538819,
 '301000': 0.06493320807756792,
 '600000': -3.2612403229395066,
 '601001': -3.0297888699030144,
 '605001': 0.29949676278332527}